In [2]:
import os
import pandas as pd

# 设置你的数据集根目录
base_data_path = '/Volumes/YuLin/no_blur_dataset/images'

# 存储训练集、验证集和（可选的）测试集的数据
train_data = []
val_data = []
test_data = [] # 暂时为空，如果你想从val中拆分，后面会填充

# 遍历所有类别文件夹
for class_name in os.listdir(base_data_path):
    class_path = os.path.join(base_data_path, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    num_images = len(images)

    # 假设你希望每类有固定的训练和验证/测试比例
    # 这里我们简化处理，假设你原始的文件夹划分就是你想要的最终划分
    # 即，如果图片来自原先的train文件夹，就放入train.csv
    # 如果图片来自原先的val文件夹，就放入val.csv (或从val中再拆分出test)

    # 由于你已经将所有图片复制到一个images文件夹下
    # 我们需要一种方式来判断这些图片是来自原始的训练集还是验证集
    # 最直接的方法是，在复制时就记录下来，或者在文件名中加入标识
    # 如果你没有这么做，我将假设你将原始的训练集图片放入训练CSV，验证集图片放入验证CSV
    # 这就需要你遍历原始的train和val目录来构建这些列表

    # 为了简化，我将提供一个通用的方法来生成所有图片的列表，
    # 之后你需要根据你的原始数据集的实际情况来分配它们到训练/验证/测试集。

    # 遍历当前类别下的所有图片
    for img_name in images:
        relative_path = os.path.join(class_name, img_name)
        # 这里你需要根据你的原始数据来源判断是训练集还是验证集
        # 这是一个示例，你需要根据你的实际情况来修改逻辑
        # 比如，你可以提前加载一个映射表，说明哪些文件属于训练，哪些属于验证
        # 或者在复制图片时，就将属于train的图片复制到一个临时目录，再统一处理

        # 假设我们现在就遍历所有图片，然后随机分配（这在Mini-ImageNet中不常用，因为它通常有预设的类别划分）
        # 更标准的做法是，你得知道哪些类别是训练集类别，哪些是验证集类别。
        # 如果你希望的是将你原始的训练集和验证集分别生成CSV：

        # --------------------------------------------------------------------------------------------------------------------------------
        # 方案一：根据原始的 train/val 文件夹来生成
        # --------------------------------------------------------------------------------------------------------------------------------
        # 这种方案需要你重新遍历你原始的 train 和 val 目录，而不是新的 miniImageNet_formatted/images 目录
        # 否则，你无法知道一个图片是来自原始的训练集还是验证集。

# --- 方案一代码 ---
# 确保你回到你的原始数据集路径
original_base_path = '/Volumes/YuLin/no_blur_dataset/' # 修改为你的原始数据集路径

# 创建存放 CSV 的目录
output_csv_dir = '/Volumes/YuLin/no_blur_dataset/splits/'
os.makedirs(output_csv_dir, exist_ok=True)

train_rows = []
val_rows = []
test_rows = [] # 用于从val中进一步拆分

# 处理训练集
print("Processing training set...")
train_source_path = os.path.join(original_base_path, 'train')
for class_name in os.listdir(train_source_path):
    class_dir = os.path.join(train_source_path, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            if img_name.endswith(('.jpg', '.jpeg', '.png')):
                # 文件名在 CSV 中应该是相对路径
                relative_path = os.path.join(class_name, img_name)
                train_rows.append({'filename': relative_path, 'label': class_name})

# 处理验证集 (并可选地拆分为验证和测试)
print("Processing validation set...")
val_source_path = os.path.join(original_base_path, 'val')
all_val_images = []
for class_name in os.listdir(val_source_path):
    class_dir = os.path.join(val_source_path, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            if img_name.endswith(('.jpg', '.jpeg', '.png')):
                relative_path = os.path.join(class_name, img_name)
                all_val_images.append({'filename': relative_path, 'label': class_name})

# 如果你希望从验证集中再拆分出测试集
# Mini-ImageNet 通常会有 64个训练类，16个验证类，20个测试类
# 你的原始数据集类别数可能不是这样，所以这里我们按比例拆分
# 假设你希望从 val 中取出 50% 作为新的 validation，50% 作为新的 test
# 或者更常见的做法是，你的原始 `val` 文件夹就是 Mini-ImageNet 格式的 `val`，
# 那么就不需要额外拆分 `test`。

# 假设你的 `val` 文件夹就是 Mini-ImageNet 格式的 `val` 集合
val_rows = all_val_images
test_rows = [] # 保持为空，或者你可以手动将某个子集指定为测试集

# 如果你需要从 val 中拆分 test，则取消注释下面的代码块，并调整比例
# from sklearn.model_selection import train_test_split
# if len(all_val_images) > 0:
#     # 示例：从 val 中拆分 50% 作为 test
#     val_rows, test_rows = train_test_split(all_val_images, test_size=0.5, random_state=42, stratify=[r['label'] for r in all_val_images])
#     print(f"Split validation set: {len(val_rows)} for new validation, {len(test_rows)} for new test.")
# else:
#     print("No images found in the original validation set.")


# 创建 DataFrame
train_df = pd.DataFrame(train_rows)
val_df = pd.DataFrame(val_rows)
test_df = pd.DataFrame(test_rows) # 如果没有测试集，这将是空 DataFrame

# 保存为 CSV 文件
train_df.to_csv(os.path.join(output_csv_dir, 'train.csv'), index=False)
val_df.to_csv(os.path.join(output_csv_dir, 'val.csv'), index=False)

# 如果你生成了测试集，也保存它
if not test_df.empty:
    test_df.to_csv(os.path.join(output_csv_dir, 'test.csv'), index=False)
    print(f"Generated train.csv ({len(train_df)} entries), val.csv ({len(val_df)} entries), and test.csv ({len(test_df)} entries) successfully.")
else:
    print(f"Generated train.csv ({len(train_df)} entries) and val.csv ({len(val_df)} entries) successfully. No test.csv generated.")

print("CSV files saved to:", output_csv_dir)

# --------------------------------------------------------------------------------------------------------------------------------
# 方案二：如果你的 `miniImageNet_formatted/images/` 已经包含所有图片，并且你希望在此基础上进行随机划分
# 这种方案可能不适用于 Mini-ImageNet 严格的类别划分（64, 16, 20），但对于快速创建自己的小样本数据集很有用。
# --- 方案二代码 (不推荐用于严格Mini-ImageNet标准，但可作为快速方案) ---
# import os
# import pandas as pd
# from sklearn.model_selection import train_test_split
#
# # 设置你的数据集根目录 (新格式的images目录)
# base_data_path = 'miniImageNet_formatted/images/'
# output_csv_dir = 'miniImageNet_formatted/splits/'
# os.makedirs(output_csv_dir, exist_ok=True)
#
# all_images = []
# for class_name in os.listdir(base_data_path):
#     class_path = os.path.join(base_data_path, class_name)
#     if not os.path.isdir(class_path):
#         continue
#     for img_name in os.listdir(class_path):
#         if img_name.endswith(('.jpg', '.jpeg', '.png')):
#             relative_path = os.path.join(class_name, img_name)
#             all_images.append({'filename': relative_path, 'label': class_name})
#
# all_df = pd.DataFrame(all_images)
#
# # 按照类别进行分层抽样，确保每个类别在训练/验证/测试集中都有代表
# # Mini-ImageNet 通常是按类别划分的，而不是随机抽样图片
# # 这里假设你希望随机抽样图片，如果需要按类别划分，则逻辑会更复杂
# train_val_df, test_df = train_test_split(all_df, test_size=0.2, random_state=42, stratify=all_df['label'])
# train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42, stratify=train_val_df['label']) # 0.25 of 0.8 is 0.2 of total
#
# train_df.to_csv(os.path.join(output_csv_dir, 'train.csv'), index=False)
# val_df.to_csv(os.path.join(output_csv_dir, 'val.csv'), index=False)
# test_df.to_csv(os.path.join(output_csv_dir, 'test.csv'), index=False)
# print(f"Generated train.csv ({len(train_df)} entries), val.csv ({len(val_df)} entries), and test.csv ({len(test_df)} entries) successfully.")
# --------------------------------------------------------------------------------------------------------------------------------

Processing training set...
Processing validation set...
Generated train.csv (8124 entries) and val.csv (1753 entries) successfully. No test.csv generated.
CSV files saved to: /Volumes/YuLin/no_blur_dataset/splits/
